# DIRECCIÓN DE LA GESTIÓN DEL CONOCIMIENTO Y LA INNOVACIÓN

# Área de Modelos Sanitarios

## ALGORITMO CITACIÓN CONTINGENCIAS COMUNES

## Objetivo

El objetivo del algoritmo de referencia es clasificar en entrada la indicación de cita de un proceso de incapacidad temporal por contingencia comunes. Para ello sólo contempla variables demográficas donde no es necesaria la citación del paciente, permitiendo así optimizar recusos humanos médicos y administrativos, asi como disminución de gastos de desplazamiento.

## Preprocesado de datos

Las variables que integran el algoritmo son:episodio (EPISODIO), edad (EDAD), código C.N.O. (CNO), recaida (REC), pago (PAGO), contrato (CONTRAT). La variable objetivo es si hay o no valoración (VAL).

Todas las variable son cualitativas a excepción de EPI y EDAD. Las características y códigos de variables se recogen en el Anexo I.

Debido a la ejecución de varias estratégias de modelado, se ha tenido que prescindir de aquellos valores perdidos o poco representativos de la muestra inicial, de tal modo tenemos que:

- Se han eliminado aquellos procesos que tengan nueve o más episodios.
- En la variable EDAD se ha tenido en cuenta el rango 18:78.
- No se han tenido en cuenta variables explicativas importantes como número de hijos, estado civil y nivel de estudios por la gran cantidad de valores perdidos que hacian perder potencia al algoritmo. 

Para la selección de variables se han tenido en cuenta análisis previos univariantes, análisis principal de componentes y eliminacion de características recursivas, resultando el análisis discriminante como el mejor.

## Estrategias de modelado

Incialmente se ha tendio en cuenta varias estrategias como árboles de decisión, maquinas de soporte vectorial, Naive-Bayes, redes neuronales y técnicas de clustering, optando por la regresión logística binaria como mejor opción.

La gran cantidad de registros en la CIE9 hacia no converger al modelo por lo que se optó por no contar con la variable "diagnóstico" que a priori parecía indispensable. 

## Desarrollo

Diariamente se ejecutará el código de predicción con los nuevos procesos del día, obteniendo la indicación de cita correspondiente. Posteriormente la Dirección Médica podrá modificar la indicación incial por razones gestoras. Finalmente los datos iniciales de carga de procesos del día se cargaran en el modelo (aprendizaje)con las modificaciones que se hayan hecho (Dirección Médica)a efectos de entrenar de nuevo el algoritmo para ganar precisión en métricas de evaluación y por lo tanto en resultados. 

La precisión global obtenida es del 80,12%.

## Necesidades a futuro

La gran cantidad de valores de perdidos en algunas variables por subregistro y la falta de validación de algunos campos de datos en el e-sanitario +, hace que el modelo pierda precisión al reducir variables explicativas que podrían aportar más robustez al mismo. Es por ello que se deberían planificar las estrategias oportunas para estas deficiencias, asi como otras que afectan a modelos futuros previstos.

# CÓDIGO ALGORITMO

In [51]:
# IMPORTACIÓN DE LIBRERÍAS NECESARIAS
import pandas as pd
import numpy as np
import sklearn.metrics as metrics
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

In [53]:
# CARGA DE DATOS
base=pd.read_csv('ENEROsindgt.csv', sep=';', encoding='latin-1')
base.head()

,EPI,EDAD,CNO,REC,PAGO,CONTRAT,VAL
0,1,49,2,1,1,1,2
1,4,52,2,1,1,1,1
2,1,48,2,1,1,1,1
3,6,41,2,1,1,1,2
4,6,32,2,1,1,1,2


In [32]:
# SLICING DE DATASET
data=base.iloc[:, 0:6]
data
target=base.iloc[:, [6]]

In [33]:
# GENERACIÓN Y ENTRENAMIENTO DEL MODELO ((fit_intercept=True, penalty='l2', tol=1e-5, 
#C=0.8, solver='lbfgs', max_iter=60,warm_start=True))
X = np.array(data)
y = np.array(target)
X.shape
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
import warnings 
warnings.filterwarnings("ignore")
model2 = linear_model.LogisticRegression()
model2.fit(X,y)
predictions = model2.predict(X)
model2.score(X,y)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=161, random_state=seed)
cv_results = model_selection.cross_val_score(model2, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)
predictions = model2.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

Logistic Regression: 0.800716 (0.034613)
0.8016152412507765
[[ 284  910]
 [  48 3587]]
             precision    recall  f1-score   support

          1       0.86      0.24      0.37      1194
          2       0.80      0.99      0.88      3635

avg / total       0.81      0.80      0.76      4829



In [57]:
# CARGA NUEVOS DATOS
Nuevos_datos=pd.read_csv('pc.csv', sep=';', encoding='latin-1')
Nuevos_datos.head()

,EPI,EDAD,CNO,REC,PAGO,CONTRAT,VAL
0,1,49,2,1,1,1,2
1,1,48,2,1,1,1,1
2,6,32,2,1,1,1,2
3,8,60,2,1,1,1,2
4,2,44,2,1,1,1,2


In [45]:
# EJECUCIÓN DEL MODELO CON NUEVOS DATOS
R=model2.predict(Nuevos_datos)
R

array([2, 2, 2, ..., 2, 2, 1], dtype=int64)

In [46]:
RESULTADO = pd.DataFrame(R) 
print(pd.value_counts(RESULTADO[0], sort = True))

2    12439
1      974
Name: 0, dtype: int64


In [22]:
# CARGA DE RESULTADOS EN DIRECTORIO
RESULTADO.to_csv("RESULTADO.csv")

#  Anexo I

CNO
	Frecuencia	Porcentaje	Porcentaje válido	Porcentaje acumulado
Válido	1	343	1,4	1,4	1,4
	2	2394	9,9	9,9	11,3
	3	2680	11,1	11,1	22,4
	4	3249	13,5	13,5	35,9
	5	5054	20,9	20,9	56,8
	6	176	,7	,7	57,6
	7	3452	14,3	14,3	71,9
	8	2335	9,7	9,7	81,5
	9	4460	18,5	18,5	100,0
	Total	24143	100,0	100,0	
